## Descripción del proyecto
La compañía Sweet Lift Taxi ha recopilado datos históricos sobre pedidos de taxis en los aeropuertos. Para atraer a más conductores durante las horas pico, necesitamos predecir la cantidad de pedidos de taxis para la próxima hora. Construye un modelo para dicha predicción.

La métrica RECM en el conjunto de prueba no debe ser superior a 48.

In [1]:
# Librerías estándar
import time

# Manejo de datos
import pandas as pd
import numpy as np

# Preprocesamiento y división de datos
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import lightgbm as lgb
import xgboost as xgb

In [8]:
# Cargamos los datos y hacemos que la primera columna sea index
raw_taxi = pd.read_csv('../raw/taxi.csv', index_col=[0], parse_dates=[0])

raw_taxi.sort_index(inplace=True)
raw_taxi = raw_taxi.resample('1h').sum() # reformulamos la serie temporal a 1h

raw_taxi.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4416 entries, 2018-03-01 00:00:00 to 2018-08-31 23:00:00
Freq: h
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   num_orders  4416 non-null   int64
dtypes: int64(1)
memory usage: 69.0 KB


In [7]:
#Enriquecimiento de datos: Separamos la data para facilitar el analisis.

raw_taxi['year'] = raw_taxi.index.year
raw_taxi['mounth'] = raw_taxi.index.month
raw_taxi['day'] = raw_taxi.index.day
raw_taxi['day_of_week'] = raw_taxi.index.dayofweek
raw_taxi.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4416 entries, 2018-03-01 00:00:00 to 2018-08-31 23:00:00
Freq: h
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   num_orders   4416 non-null   int64
 1   year         4416 non-null   int32
 2   mounth       4416 non-null   int32
 3   day          4416 non-null   int32
 4   day_of_week  4416 non-null   int32
dtypes: int32(4), int64(1)
memory usage: 138.0 KB


In [9]:
# Enriquecimiento de datos optimizado
def make_features(data, max_lag, rolling_mean_size):
    """
    Genera variables de lag y media móvil para enriquecer el conjunto de datos.
    
    Parámetros:
    - data: pd.DataFrame con la columna 'num_orders'.
    - max_lag: int, número máximo de variables de lag a generar.
    - rolling_mean_size: int, ventana de la media móvil.

    Retorna:
    - pd.DataFrame con nuevas características y sin valores NaN.
    """
    # Generar variables de lag de forma vectorizada
    lags = {f'lag_{lag}': data['num_orders'].shift(lag) for lag in range(1, max_lag + 1)}
    lag_df = pd.DataFrame(lags)

    # Calcular la media móvil
    rolling_mean = data['num_orders'].shift(1).rolling(rolling_mean_size).mean()

    # Concatenar las nuevas características con el conjunto de datos original
    enriched_data = pd.concat([data, lag_df], axis=1)
    enriched_data['rolling_mean'] = rolling_mean

    # Eliminar filas con valores NaN generados por los lags y la media móvil
    enriched_data = enriched_data.dropna()

    return enriched_data

In [10]:
#Enriquecemos la data agregando una media movil de 20 y 40 ultimos valores en desface
data_taxi = make_features(raw_taxi, 40, 20)
data_taxi

,num_orders,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,...,lag_32,lag_33,lag_34,lag_35,lag_36,lag_37,lag_38,lag_39,lag_40,rolling_mean
datetime,,,,,,,,,,,,,,,,,,,,,
2018-03-02 16:00:00,82,59.0,44.0,45.0,36.0,91.0,54.0,45.0,46.0,7.0,...,34.0,15.0,12.0,6.0,43.0,66.0,71.0,85.0,124.0,55.80
2018-03-02 17:00:00,83,82.0,59.0,44.0,45.0,36.0,91.0,54.0,45.0,46.0,...,69.0,34.0,15.0,12.0,6.0,43.0,66.0,71.0,85.0,56.85
2018-03-02 18:00:00,89,83.0,82.0,59.0,44.0,45.0,36.0,91.0,54.0,45.0,...,64.0,69.0,34.0,15.0,12.0,6.0,43.0,66.0,71.0,57.70
2018-03-02 19:00:00,49,89.0,83.0,82.0,59.0,44.0,45.0,36.0,91.0,54.0,...,96.0,64.0,69.0,34.0,15.0,12.0,6.0,43.0,66.0,56.50
2018-03-02 20:00:00,93,49.0,89.0,83.0,82.0,59.0,44.0,45.0,36.0,91.0,...,30.0,96.0,64.0,69.0,34.0,15.0,12.0,6.0,43.0,56.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-08-31 19:00:00,136,207.0,217.0,197.0,116.0,133.0,166.0,136.0,167.0,191.0,...,156.0,227.0,226.0,166.0,39.0,13.0,56.0,186.0,193.0,169.35
2018-08-31 20:00:00,154,136.0,207.0,217.0,197.0,116.0,133.0,166.0,136.0,167.0,...,173.0,156.0,227.0,226.0,166.0,39.0,13.0,56.0,186.0,163.85
2018-08-31 21:00:00,159,154.0,136.0,207.0,217.0,197.0,116.0,133.0,166.0,136.0,...,154.0,173.0,156.0,227.0,226.0,166.0,39.0,13.0,56.0,157.75


#### Analiza los datos.
Entrena diferentes modelos con diferentes hiperparámetros. La muestra de prueba debe ser el 10% del conjunto de datos inicial.

In [11]:
# Dividimos los datos en conjuntos de entrenamiento y prueba
# Usamos shuffle=False para mantener el orden temporal de los datos
train, test = train_test_split(data_taxi, test_size=0.1, shuffle=False)

# Separar características (features) y el objetivo (target) en el conjunto de entrenamiento
features_train = train.drop('num_orders', axis=1) 
target_train = train['num_orders']  

# Separar características y el objetivo en el conjunto de prueba
features_test = test.drop('num_orders', axis=1) 
target_test = test['num_orders']

In [ ]:
# Iniciamos el entrenamiento de los Modelos de Machine Learning
# Entrenamiento y evaluación del Modelo de Regresion Lineal

start_time = time.time()  # Iniciamos el cronómetro para medir el tiempo de ejecución

# Instanciamos y entrenamos el modelo
lr_model = LinearRegression()
lr_model.fit(features_train, target_train)

# Realizamos predicciones sobre el conjunto de prueba
lr_pred = lr_model.predict(features_test)

# Calculamos el RMSE para evaluar el modelo
lr_rmse = np.sqrt(mean_squared_error(target_test, lr_pred))

# Registramos el tiempo total de ejecución
lr_time = time.time() - start_time

# Mostramos los resultados
print(f"RMSE para Regresión Lineal: {lr_rmse}")
print(f"Tiempo de ejecución del modelo Regresión Lineal: {lr_time:.4f} segundos")

RMSE para Regresión Lineal: 44.78570421513906
Tiempo de ejecución del modelo Regresión Lineal: 0.2953 segundos


In [15]:
# Entrenamiento y evaluación del modelo Árbol de Decisión
start_time = time.time()  # Iniciamos el cronómetro para medir el tiempo de ejecución

# Instanciamos y entrenamos el modelo
dt_model = DecisionTreeRegressor()
dt_model.fit(features_train, target_train)

# Realizamos predicciones sobre el conjunto de prueba
dt_pred = dt_model.predict(features_test)

# Calculamos el RMSE para evaluar el modelo
dt_rmse = np.sqrt(mean_squared_error(target_test, dt_pred))

# Registramos el tiempo total de ejecución
dt_time = time.time() - start_time

# Mostramos los resultados
print(f"RMSE para Árbol de Decisión: {dt_rmse}")
print(f"Tiempo de ejecución del modelo Árbol de Decisión: {dt_time:.4f} segundos")

RMSE para Árbol de Decisión: 60.167479345376066
Tiempo de ejecución del modelo Árbol de Decisión: 0.1692 segundos


In [17]:
# Entrenamiento y evaluación del modelo Bosque Aleatorio
start_time = time.time()  # Iniciamos el cronómetro para medir el tiempo de ejecución

# Instanciamos y entrenamos el modelo
rf_model = RandomForestRegressor()
rf_model.fit(features_train, target_train)

# Realizamos predicciones sobre el conjunto de prueba
rf_pred = rf_model.predict(features_test)

# Calculamos el RMSE para evaluar el modelo
rf_rmse = np.sqrt(mean_squared_error(target_test, rf_pred))

# Registramos el tiempo total de ejecución
rf_time = time.time() - start_time

# Mostramos los resultados
print(f"RMSE para Bosque Aleatorio: {rf_rmse}")
print(f"Tiempo de ejecución del modelo Bosque Aleatorio: {rf_time:.4f} segundos")

RMSE para Bosque Aleatorio: 43.63364084068276
Tiempo de ejecución del modelo Bosque Aleatorio: 11.2732 segundos


In [18]:
# Entrenamiento y evaluación del modelo LightGBM
start_time = time.time()  # Iniciamos el cronómetro para medir el tiempo de ejecución

# Instanciamos y entrenamos el modelo
lgb_model = lgb.LGBMRegressor(
    random_state=42, 
    n_estimators=100, 
    learning_rate=0.1, 
    force_col_wise=True
)
lgb_model.fit(features_train, target_train)

# Realizamos predicciones sobre el conjunto de prueba
lgb_pred = lgb_model.predict(features_test)

# Calculamos el RMSE para evaluar el modelo
lgb_rmse = np.sqrt(mean_squared_error(target_test, lgb_pred))

# Registramos el tiempo total de ejecución
lgb_time = time.time() - start_time

# Mostramos los resultados
print(f"RMSE para LightGBM: {lgb_rmse}")
print(f"Tiempo de ejecución del modelo LightGBM: {lgb_time:.4f} segundos")

[LightGBM] [Info] Total Bins 7833
[LightGBM] [Info] Number of data points in the train set: 3938, number of used features: 41
[LightGBM] [Info] Start training from score 78.597765
RMSE para LightGBM: 43.786771234399225
Tiempo de ejecución del modelo LightGBM: 0.3292 segundos


In [19]:
# Entrenamiento y evaluación del modelo XGBoost
start_time = time.time()  # Iniciamos el cronómetro para medir el tiempo de ejecución

# Instanciamos y entrenamos el modelo
xgb_model = xgb.XGBRegressor(
    random_state=42, 
    n_estimators=100, 
    learning_rate=0.1
)
xgb_model.fit(features_train, target_train)

# Realizamos predicciones sobre el conjunto de prueba
xgb_pred = xgb_model.predict(features_test)

# Calculamos el RMSE para evaluar el modelo
xgb_rmse = np.sqrt(mean_squared_error(target_test, xgb_pred))

# Registramos el tiempo total de ejecución
xgb_time = time.time() - start_time

# Mostramos los resultados
print(f"RMSE para XGBoost: {xgb_rmse}")
print(f"Tiempo de ejecución del modelo XGBoost: {xgb_time:.4f} segundos")


RMSE para XGBoost: 42.971296566823256
Tiempo de ejecución del modelo XGBoost: 3.2695 segundos


#### Comparacion de resultados de modelos usados

In [21]:
# Consolidamos los resultados de los modelos en un DataFrame
results = pd.DataFrame({
    'Model': ['Regresión Lineal', 'Árbol de Decisión', 'Bosque Aleatorio', 'LightGBM', 'XGBoost'],
    'RMSE': [lr_rmse, dt_rmse, rf_rmse, lgb_rmse, xgb_rmse],
    'Execution Time (s)': [lr_time, dt_time, rf_time, lgb_time, xgb_time]
})

# Ordenamos los resultados por RMSE en orden ascendente para identificar el modelo más preciso
results = results.sort_values(by='RMSE', ascending=True).reset_index(drop=True)

# Mostramos los resultados ordenados
results


,Model,RMSE,Execution Time (s)
0,XGBoost,42.971297,3.269476
1,Bosque Aleatorio,43.633641,11.273212
2,LightGBM,43.786771,0.329190
3,Regresión Lineal,44.785704,0.295269
4,Árbol de Decisión,60.167479,0.169155


In [24]:
# Reordenamos en funcion al tiempo de ejecucion para ver la velocidad de cada modelo
results.sort_values('Execution Time (s)', ascending=True)

,Model,RMSE,Execution Time (s)
4,Árbol de Decisión,60.167479,0.169155
3,Regresión Lineal,44.785704,0.295269
2,LightGBM,43.786771,0.329190
0,XGBoost,42.971297,3.269476
1,Bosque Aleatorio,43.633641,11.273212


### Conclusiones
- De los cinco modelos evaluados, cuatro lograron un RMSE inferior a 45, lo que indica un buen desempeño general en la predicción de la serie temporal.
- La inclusión de características desfasadas ('lag') tuvo un impacto significativo en la reducción del RMSE. Sin embargo, en el caso del Árbol de Decisión, el RMSE no pudo bajar de 60 sin extender los valores de 'lag' hasta 100, lo que incrementaba el tamaño de los datos de manera considerable y afectaba el rendimiento del modelo.
- Aunque las características basadas en medias móviles aportaron mejoras en el RMSE, su impacto fue menor en comparación con las características 'lag'.
- Los modelos LightGBM y XGBoost mostraron el mejor desempeño en este caso, equilibrando precisión y tiempo de ejecución
- Si bien inicialmente la Regresión Lineal ofrecía resultados competitivos, su precisión disminuyó en comparación con los modelos más avanzados al añadir más características 'lag'. Sin embargo, sigue siendo la opción más rápida para obtener resultados aceptables y rapidos.  